In [ ]:

import rasterio
import matplotlib.pyplot as plt
import os
from matplotlib.colors import ListedColormap
import re
import csv
from skimage import io
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from scipy.ndimage import binary_dilation, uniform_filter

In [ ]:
tile = '10SGD'
location = 'Isla_Vista_Kelp_2018'
path = os.path.join(r'/mnt/c/Users/attic/HLS_Kelp/imagery',location,tile)
granule ='HLS.S30.T10SGD.2018257T183949.v2.0'
files = os.listdir(os.path.join(path,granule))
metadata_file = [f for f in files if re.search(r'metadata\.csv$', f)]
if metadata_file :
    with open(os.path.join(path,granule, metadata_file[0]), mode='r') as file:
        csv_reader = csv.reader(file)
        keys = next(csv_reader)  
        values = next(csv_reader) 
    metadata = dict(zip(keys, values))
urls = metadata['data_vis_url']
img_urls = urls.strip("[]").replace("'", "").split(", ")
print(img_urls)
response = requests.get(img_urls[0])
img = Image.open(BytesIO(response.content))
img.show()


In [ ]:
path = r'/mnt/c/Users/attic/HLS_Kelp/imagery/10SGD'
files = os.listdir(path)
tif_files = [file for file in files if re.search(r'\.tif$', file, re.IGNORECASE)]
sorted_tif = np.sort(tif_files)
training_data = []
cmap = ListedColormap(['red', 'blue', 'green'])
for i, file in enumerate(sorted_tif):
    with rasterio.open(os.path.join(path,file)) as src:#path, file)) as src:
        mesma_img = src.read(6) 
        kelp_img = src.read(5) 
        green_band = src.read(2)
        green_normalized =green_band/(50*green_band.max())
        print(file)
        kelp_dilation = np.ones((15,15))
        kelp_neighborhood_min = 5
        min_kelp_count = 4
        kelp_mask  = []
        mesma_img = np.where(mesma_img == 0, np.nan, mesma_img)
        mesma_img = np.where(mesma_img >= 100, 100, mesma_img)
        full_vis = np.where(green_normalized == 0, np.nan, green_normalized)
        kelp_dilated = np.where(kelp_img == 0, True, False) #This is expanding hte kelp_mask so the TF is reversed
        kelp_count = uniform_filter(kelp_dilated.astype(int),size=kelp_neighborhood_min)
        kelp_dilated = np.where(((~kelp_dilated) | (kelp_count >= min_kelp_count)), 0, 1) #If theres no kelp, or the kelp count is <=4, set pixel == false
        kelp_dilated = binary_dilation(kelp_dilated,structure=kelp_dilation) 
        kelp_dilated = np.where(kelp_dilated == 0, np.nan, green_normalized )
        plt.figure(figsize=(15, 15))
        plt.imshow(full_vis[2500:3500,2500:3500]/10, alpha=1)
        plt.imshow(kelp_dilated[2500:3500,2500:3500], cmap = "Greens", alpha=1)

        plt.imshow(mesma_img[2500:3500,2500:3500], cmap ='Reds',alpha=1)
        plt.colorbar()
        plt.show()  
# plt.figure(figsize=(6, 6))
# plt.imshow(training_img[6,:], cmap='gray')
# plt.show()  